In [1]:
import torch
import numpy as np
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

from model import KeywordSpottingModel
from data_loader import load_speech_commands_dataset
from utils import set_memory_GB,print_model_size, log_to_file
from augmentations import add_time_shift_noise_and_align, add_noise
from train import trainig_loop




/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/selective_scan_interface.py:164: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, xz, conv1d_weight, conv1d_bias, x_proj_weight, delta_proj_weight,
/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/selective_scan_interface.py:240: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, dout):
/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/triton/layer_norm.py:986: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(
/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/triton/layer_norm.py:1045: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type=

In [2]:
torch.cuda.is_available()

True

In [3]:
set_memory_GB(1)

Memory fraction set to 0.022458079576498518
Memory fraction in GB: 1.0


In [4]:
train_ds, val_ds, test_ds, info = load_speech_commands_dataset()

2024-08-14 13:57:30.860474: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-14 13:57:30.866588: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-14 13:57:30.881647: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-14 13:57:30.907765: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-14 13:57:30.915287: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attemptin

In [5]:
# maintain seed for repructablity
np.seed = 42
# tf.random.set_seed(42)
torch.manual_seed(0)

In [6]:
label_names = info.features['label'].names
print(label_names)

['down', 'go', 'left', 'no', 'off', 'on', 'right', 'stop', 'up', 'yes', '_silence_', '_unknown_']


In [7]:
augmentations = [
    lambda x: add_time_shift_noise_and_align(x),
    lambda x:add_noise(x,noise_level = 0.01)
]

In [18]:

from librosa.feature import mfcc
from torch.utils.data import Dataset
# Define the dataset adapter:
class TFDatasetAdapter_with_cls(Dataset):
    def __init__(self, tf_dataset, fixed_length, n_mfcc, n_fft, hop_length, n_mels, augmentations=None, cls_token=torch.zeros(13)):
        self.tf_dataset = tf_dataset
        self.data = list(tf_dataset)
        self.fixed_length = fixed_length
        self.n_mfcc = n_mfcc
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.n_mels = n_mels
        self.augmentations = augmentations
        self.cls_token = cls_token

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        audio, label = self.data[idx]
        audio = audio.numpy()

        # Ensure the audio tensor has the correct shape (1D array)
        if audio.ndim > 1:
            audio = np.squeeze(audio)
            
        # Apply augmentations if any
        if self.augmentations:
            for aug in self.augmentations:
                audio = aug(audio)

        # Pad or trim the audio to the fixed length
        if len(audio) < self.fixed_length:
            audio = np.pad(audio, (0, self.fixed_length - len(audio)), mode='constant')
        else:
            audio = audio[:self.fixed_length]

        # Create MFCCs from an audio tensor using Librosa.
        audio = audio.astype(np.float32)
        MFCC = mfcc(y=audio, sr=16000, n_mfcc=self.n_mfcc, n_fft=self.n_fft, hop_length=self.hop_length, n_mels=self.n_mels)

        # Remove extra dimension if it exists
        if MFCC.ndim == 3:
            MFCC = MFCC.squeeze(-1)

        # Add a CLS token to the MFCCs
        MFCC = torch.tensor(MFCC, dtype=torch.float32)
        MFCC = self.add_cls_token(MFCC)

        label = torch.tensor(label.numpy(), dtype=torch.long)

        return MFCC, label
    
    def add_cls_token(self, x):
        # x: input tensor of shape (input_dim, time_dim)
        cls_token = self.cls_token.unsqueeze(1)  # Shape: (input_dim, 1)
        x_with_cls = torch.cat([cls_token, x], dim=1)  # Concatenate along the time dimension
        return x_with_cls


In [19]:
# Convert the TFDS dataset to a PyTorch Dataset
fixed_length = 16000
n_mfcc = 13
n_fft = 400
hop_length = 160
n_mels = 40
pytorch_train_dataset = TFDatasetAdapter_with_cls(train_ds, fixed_length, n_mfcc, n_fft, hop_length, n_mels, augmentations)
pytorch_val_dataset = TFDatasetAdapter_with_cls(val_ds, fixed_length, n_mfcc, n_fft, hop_length, n_mels, augmentations=None)

In [20]:
# Create a DataLoader to feed the data into the model
batch_size = 32
train_loader = DataLoader(pytorch_train_dataset, batch_size=batch_size, shuffle=True,num_workers=4,prefetch_factor=2)
val_loader = DataLoader(pytorch_val_dataset, batch_size=batch_size, shuffle=False,num_workers=4,prefetch_factor=2)

In [21]:
for audio, label in train_loader:
    print(audio.shape, label.shape)
    break


torch.Size([32, 13, 102]) torch.Size([32])


# Compute model size

In [25]:
import torch.nn as nn
from mamba_ssm import Mamba

# Define model architecture
class KeywordSpottingModel_witch_cls(nn.Module):
    def __init__(self, input_dim, d_model, d_state, d_conv, expand, label_names):
        super(KeywordSpottingModel_witch_cls, self).__init__()
        self.proj = nn.Linear(input_dim, d_model)  # Initial projection layer
        self.mamba = Mamba(
            d_model=d_model,
            d_state=d_state,
            d_conv=d_conv,
            expand=expand
        )
        self.fc = nn.Linear(d_model, len(label_names))  # Output layer

        self.dropout = nn.Dropout(0.5)  # Dropout layer with a dropout rate of 0.5


    def forward(self, x):
        x = x.permute(0, 2, 1)  # Reshape to [batch_size, num_frames, num_mfcc]
        x = self.proj(x)  # Project input to d_model dimension
        x = x.permute(0, 2, 1)  # Transpose to [batch_size, d_model, num_frames] for Mamba
        x = self.mamba(x)
        x = self.dropout(x)  # Apply dropout after Mamba
        x_cls = x[:, 0]
        x = self.fc(x_cls)  # Apply the output layer
        return x

In [26]:
# Initialize model, loss function, and optimizer
input_dim = 13  # Number of MFCC features
d_model = 102  # Number of frames
d_state = 16
d_conv = 4
expand = 2

model = KeywordSpottingModel(input_dim=input_dim, d_model=d_model, d_state=d_state, d_conv=d_conv, expand=expand, label_names=info.features['label'].names).to("cuda")


# Training loop

# With L2 regulariztion AND Droput layer

In [35]:
# Initialize model, loss function, and optimizer
input_dim = 13  # Number of MFCC features
d_model = 102  # Number of frames
d_state = 16
d_conv = 4
expand = 2

model = KeywordSpottingModel_witch_cls(input_dim=input_dim, d_model=d_model, d_state=d_state, d_conv=d_conv, expand=expand, label_names=label_names).to("cuda")
criterion = nn.CrossEntropyLoss()
base_optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5) # weight_decay for L2 regulariztopn

from torch_optimizer import Lookahead

optimizer = Lookahead(base_optimizer, k=5, alpha=0.5)  # Wrap around Adam


In [36]:
import torch.optim as optim
# Adding learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

In [37]:
import torch
from utils import log_to_file, EarlyStopping
from tqdm import tqdm


# Initialize the early stopping object
early_stopping = EarlyStopping(patience=5, min_delta=0.001)

# Training loop
num_epochs = 70

train_accuracies = []
val_accuracies = []
train_losses = []
val_losses = []

# Log new training session
log_to_file("\n\nNew training session\n\n","trainig_log_witch_cls.txt")
# Log the model architecture
log_to_file(str(model),"trainig_log_witch_cls.txt")


for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for audio, labels in tqdm(train_loader):
        audio, labels = audio.to("cuda"), labels.to("cuda")

        # Forward pass
        outputs = model(audio)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_accuracy = 100 * correct_train / total_train
    train_loss = running_loss / len(train_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss}, Training Accuracy: {train_accuracy}%')

    train_accuracies.append(train_accuracy)
    train_losses.append(train_loss)

    # Log training metrics
    log_to_file(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%',"trainig_log_witch_cls.txt")

    # # Step the scheduler
    # scheduler.step()

    # Validation
    model.eval()
    correct_val = 0
    total_val = 0
    val_loss = 0.0

    with torch.no_grad():
        for audio, labels in val_loader:
            audio, labels = audio.to("cuda"), labels.to("cuda")
            outputs = model(audio)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_accuracy = 100 * correct_val / total_val
    val_loss_avg = val_loss / len(val_loader)
    print(f'Validation Loss: {val_loss_avg}, Validation Accuracy: {val_accuracy}%')

    val_accuracies.append(val_accuracy)
    val_losses.append(val_loss_avg)
    
    # Step the scheduler based on validation loss
    scheduler.step(val_loss_avg)
    print(f'Learning rate after epoch {epoch+1}: {scheduler.get_last_lr()}')

    
    
    # Check early stopping condition
    if early_stopping.step(val_loss/len(val_loader)):
        print(f"Stopping training at epoch {epoch+1} due to early stopping")
        break

    # Log validation metrics
    log_to_file(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss_avg:.4f}, Validation Accuracy: {val_accuracy:.2f}%',"trainig_log_witch_cls.txt")

log_to_file("Training complete.","trainig_log_witch_cls.txt")



100%|██████████████████████████████████████████████████████████████████| 2673/2673 [02:32<00:00, 17.54it/s]

Epoch 1/70, Training Loss: 7.381863319601765, Training Accuracy: 59.78762966168095%


Validation Loss: 1.5402221434478518, Validation Accuracy: 61.8590378142942%
Learning rate after epoch 1: [0.001]


100%|██████████████████████████████████████████████████████████████████| 2673/2673 [02:30<00:00, 17.80it/s]

Epoch 2/70, Training Loss: 525.659531938441, Training Accuracy: 57.46278256598566%


...

100%|██████████████████████████████████████████████████████████████████| 2673/2673 [02:30<00:00, 17.77it/s]

Epoch 41/70, Training Loss: 1.043401622567068, Training Accuracy: 67.61937060729029%


Validation Loss: 1.0075836864453327, Validation Accuracy: 68.36270045535538%
Learning rate after epoch 41: [0.001]


  4%|██▉                                                                | 115/2673 [00:08<03:10, 13.43it/s]


KeyboardInterrupt: 